In [ ]:
import json
import datetime
import pandas as pd

In [ ]:
deck_data = []
malformed_deck_data = []
match_data = []
malformed_data = []

In [ ]:
BASE_PATH = "/mnt/c/Users/chris"

In [ ]:
with open(f"{BASE_PATH}/AppData/LocalLow/Wizards Of The Coast/MTGA/Player.log", "r") as f:
    for line in f.readlines():
        if line.startswith('{"Summary":{"DeckId":') is True:
            try:
                deck_data.append(json.loads(line))
            except:
                malformed_deck_data.append(line)
        elif '"stateType": "MatchGameRoomStateType_MatchCompleted"' in line:
            try:
                match_data.append(json.loads(line))
            except:
                malformed_data.append(line)

In [ ]:
assert len(deck_data) == len(match_data)

In [ ]:
match_data_parsed = [
    {
        "game_id": match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["gameRoomConfig"]["matchId"],
        "timestamp": datetime.datetime.fromtimestamp(int(match["timestamp"]) / 1000),
        "deck_name": deck["Summary"]["Name"],
        "game_won": [
            player["teamId"] for player in match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["gameRoomConfig"]["reservedPlayers"]
            if player["playerName"] == "spantz"
        ][0] == [
            result["winningTeamId"] for result in match["matchGameRoomStateChangedEvent"]["gameRoomInfo"]["finalMatchResult"]["resultList"]
            if result["scope"] == "MatchScope_Game"
        ][0]
    } for deck, match in zip(deck_data, match_data)
]

In [ ]:
match_df = pd.DataFrame(match_data_parsed)

In [ ]:
match_df.to_csv(f"{BASE_PATH}/Desktop/mtga_match_log.csv", index=False, header=False, mode="a")

In [ ]:
import shutil
import os
import time


source_path = f"{BASE_PATH}/AppData/LocalLow/Wizards Of The Coast/MTGA/Player.log"
destination_path = f"{BASE_PATH}/AppData/LocalLow/Wizards Of The Coast/MTGA/Player_{time.time_ns()}.log"

# Ensure the source file exists
if os.path.exists(source_path):
    shutil.move(source_path, destination_path)
    print(f"File moved from {source_path} to {destination_path}")
else:
    print(f"Source file {source_path} not found.")